# Introduction

This notebook will go through the basic steps of preparing the input and calculating non-local quantities.

# Input

The first step will be the preparation of input data. For this tutorial we assume, that you have been provided with a file containing the DMFT output data. 

We will convert this file to an input file for the `LadderDGA.jl` code and generate a frequency mesh. 

You can also peak into HDF5 files using the Julia HDF5 package.

In [4]:
using HDF5
path_to_input = "/home/julian/Hamburg/lDGA_checks/triangular_test/U2_n1_b12.h5"
f = h5open(path_to_input)

LoadError: unable to determine if /home/julian/Hamburg/lDGA_checks/triangular_test/U2_n1_b12.h5 is accessible in the HDF5 format (file may not exist)

## TRIQS Conversion

If you are starting out with DMFT data from TRIQS, the `triqs_conv.jl` script in the scripts subdirectory of the `LadderDGA.jl` code will responsible for the conversion. This script also utilizes the `SparseVertex` and `EquivalencyClassesConstructor.jl` packages. 

Most of the conversion scripts are built to be called from the command line with the input given as parameters.
In Julia call parameters are stored in the `ARGS` array. Thus, in order to emulate a call with parameters in jupyter notebooks, we will modify this array befor calls to scripts.

We start by adding the path to a triqs output file and a directory for the script output (`@__DIR__` points to the directory the current script runs in) to the `ARGS` array and executing the `triqs_conv.jl` script for this input.
The input file is NOT provided with this example!

If you want to know more about a function, you can type `? function_name` in any cell to view the docstring.

In [5]:
path_to_input = "/home/julian/Hamburg/lDGA_checks/square_test/U2_n1_b12.h5"
path_to_output = "/home/julian/Hamburg/lDGA_checks/square_test"
empty!(ARGS)
push!(ARGS,path_to_input)
push!(ARGS,path_to_output)
include("../scripts/triqs_conv.jl")

┌ Info: Precompiling SparseVertex [90e5b0d8-73d4-48d2-a116-5fd8650b07f2]
└ @ Base loading.jl:1317


Generating freqency mesh with bosonic -59:59, fermionic -60:59 indices
Constructing Array
Starting Computation 3
  1.485741 seconds (139.52 k allocations: 110.950 MiB, 2.76% gc time, 21.92% compilation time)
  0.128148 seconds (60 allocations: 16.170 MiB)
  0.497274 seconds (11 allocations: 78.148 MiB)


## Config File
You will find two new files in the `path_to_output` directory. Before we can start the calculation we need to specify a configuration file which will point `LadderDGA.jl` to these files and provide all necessary parameters.

You will find an example configuration named `config.toml` in the root directory of `LadderDGA.jl`.
`U`, `mu`, `beta`, `nden` and `kGrid` should be set according to your DMFT calculation. The syntax for the `kGrid` parameter is as follows: The grid is given as a string starting with the grid name (see also `Dispersions.jl` for more information), followed by additional parameters, separated by `-`. Currently `2Dsc` and `3Dsc` are available with a single parameter for the hopping to neighbouring sites. Examples are `"2Dsc-0.25"` or `"3Dsc-0.408248"`

Remember to set the `inputDir` and `freqFile` variables to the previously generated files.

# Running the code

The `LadderDGA.jl` code consists of a collection of functions but no linear program flow. This means one has to define a script which defines the actual program flow.
Some examples can be found in the root directory of the project. Here we will go through a typical program flow, also giving us the opportunity to plot quantities at differnts steps of the calculation.
First, point the `cfg_file` variable to your edited configuration file.
We then also tell Julia to use `Plots` and `LadderDGA`.

LadderDGA.jl can be run in parallel by specifying the `path_to_source` variable. If you want to run the single core version, comment out lines `4` to `10` and uncomment line `13`.

In [6]:
cfg_file = "/home/julian/Hamburg/lDGA_checks/square_test/config.toml"
path_to_source = "/home/julian/Hamburg/LadderDGA.jl"

using Distributed
if nprocs() == 1
    addprocs(7)
    @everywhere using Pkg
    @everywhere Pkg.activate(path_to_source)
    @everywhere using LadderDGA
end

using Plots
#using LadderDGA

The next step will read the config and input files and set up all variables for you.
The `kGrids` (reduced k grid) arrays contain k grids of the sizes specified in the `Nk` array in your `config.toml`. 
This can be handy for finite size scaling, but for now we will stick with simple calculations and only use a single k grid size.
Finally we will also need to read in the DMFT quantities. `setup_LDGA` will do just that.

Note: the full grids (here stored in the kGrids array) are deprecated. They will be removed from future versions, together with most debug variables (all variables in the return list of `readConfig` after `qGridLoc`).

In [2]:
mP, sP, env, kGrids, qGridLoc, freqRed_map, freqList, freqList_min, parents, ops, nFermi, nBose, shift, base, offset = readConfig(cfg_file)
qG = kGrids[1]
νGrid, sumHelper_f, impQ_sp, impQ_ch, GImp_fft, GLoc_fft, Σ_loc, FUpDo, gImp, gLoc = setup_LDGA(qG, freqList, mP, sP, env);

┌ Info: Reading Inputs...
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/IO.jl:7
┌ Warning: TODO: check beta consistency, config <-> g0man, chi_dir <-> gamma dir
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/helpers.jl:83
┌ Info: setting usable ranges of sp and ch channel from 44:56 and 44:56 to the same range of 44:56
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/helpers.jl:135
┌ Warning: Could not find hubb.andpar and gm_wim for kinetic energy. proceding without improved bosonic sums!
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/helpers.jl:148
┌ Info: Inputs Read. Starting Computation.
│ Local susceptibilities with ranges are:
│ χLoc_sp(44:56) = 0.2411, χLoc_ch(44:56) = 0.1911 
│ sum χupup check (fit, tail sub, tail sub + fit, expected): 0.21609829006973458 ?≈? 0.22488273176943194 ?=? 0.21609829006973325 ?≈? 0.25"
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/helpers.jl:169


We are now in the position to calculate the ladder DGA self energy. The first 5 lines compute all quantities on a lattice of size 1. The resulting self energy is used to cancel out discretization errors and finite size effects in the full self energy.

In [3]:
bubbleLoc = calc_bubble(νGrid, GImp_fft, qGridLoc, mP, sP)
locQ_sp = calc_χ_trilex(impQ_sp.Γ, bubbleLoc, qGridLoc, νGrid, sumHelper_f, mP.U, mP, sP);
locQ_ch = calc_χ_trilex(impQ_ch.Γ, bubbleLoc, qGridLoc, νGrid, sumHelper_f, -mP.U, mP, sP);
Σ_ladderLoc = calc_Σ(locQ_sp, locQ_ch, bubbleLoc, GImp_fft, FUpDo, qGridLoc, sumHelper_f, mP, sP)
Σ_ladderLoc = Σ_ladderLoc .+ mP.n * mP.U/2.0;


bubble = calc_bubble(νGrid, GLoc_fft, qG, mP, sP);
nlQ_sp = calc_χ_trilex(impQ_sp.Γ, bubble, qG, νGrid, sumHelper_f, mP.U, mP, sP);
nlQ_ch = calc_χ_trilex(impQ_ch.Γ, bubble, qG, νGrid, sumHelper_f, -mP.U, mP, sP);
nlQ_sp_nλ = deepcopy(nlQ_sp)
nlQ_ch_nλ = deepcopy(nlQ_ch)
λ_sp, λ_spch  = λ_correction!(impQ_sp, impQ_ch, FUpDo, Σ_loc, Σ_ladderLoc, nlQ_sp, nlQ_ch, bubble, GLoc_fft, qG, mP, sP)

Σ_ladder = calc_Σ(nlQ_sp, nlQ_ch, bubble, GLoc_fft, FUpDo, qG, sumHelper_f, mP, sP)
Σ_ladder_corrected = Σ_ladder .- Σ_ladderLoc .+ Σ_loc[1:size(Σ_ladder,1)];

┌ Info: Computing λsp corrected χsp, using 
│   sP.χFillType = lambda_χ_fill
│    as fill value outside usable ω range. =  as fill value outside usable ω range.
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/lambdaCorrection.jl:222
┌ Warning: currently using min(usable_sp, usable_ch) = min(44:56,44:56) = 44:56 for all calculations. relax this?
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/lambdaCorrection.jl:33
┌ Info: Found usable intervals for non-local susceptibility of length 
│ sp: 44:56, length: 13
│ ch: 44:56, length: 13
│ usable: 44:56, length: 13
│ χch sum = 0.19864789730366825, rhs = 0.3013521026963317
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/lambdaCorrection.jl:47
┌ Info: found 
│   χ_min = -4.93784005901441
│   ". Looking for roots in intervall $(int)" = . Looking for roots in intervall [-4.93784005901441, 16.72882660765226]
└ @ LadderDGA /home/julian/Hamburg/LadderDGA.jl/src/lambdaCorrection.jl:214
┌ Info: Method 2 root:
│   r2 = Float64[]
└ @ LadderD

# Visualization

The main advantage of calculations in jupyter notebooks are the convenient options for data visualization. The following cells give examples for plotting in Julia

In [6]:
sh = LadderDGA.get_sum_helper(nlQ_ch.usable_ω, sP, :b)
χch_ω = LadderDGA.kintegrate(qG, nlQ_ch.χ, dim=2)[:,1]
χch_sum = real(LadderDGA.sum_freq(χch_ω[nlQ_ch.usable_ω], [1], sh, mP.β)[1])
real(impQ_ch.χ_loc + impQ_sp.χ_loc - χch_sum)

sh = LadderDGA.get_sum_helper(nlQ_sp.usable_ω, sP, :b)
χsp_ω = LadderDGA.kintegrate(qG, nlQ_sp.χ, dim=2)[:,1]
χsp_sum = real(LadderDGA.sum_freq(χsp_ω[nlQ_sp.usable_ω], [1], sh, mP.β)[1])

LoadError: UndefVarError: nlQ_ch not defined

In [7]:
χchED_ω_naive = sum(impQ_ch.χ,dims=[2,3])[:,1,1]/(mP.β^2);
χspED_ω_naive = sum(impQ_sp.χ,dims=[2,3])[:,1,1]/(mP.β^2);

reduction_prct = -0.1
χchED_ω_fit = [LadderDGA.sum_freq(impQ_ch.χ[i,:,:], [1,2], sumHelper_f, mP.β)[1,1] for i in 1:size(impQ_ch.χ,1)];
χspED_ω_fit = [LadderDGA.sum_freq(impQ_sp.χ[i,:,:], [1,2], sumHelper_f, mP.β)[1,1] for i in 1:size(impQ_sp.χ,1)];
usable_ch = LadderDGA.find_usable_interval(real(χchED_ω_fit), reduce_range_prct=reduction_prct)
usable_sp = LadderDGA.find_usable_interval(real(χspED_ω_fit), reduce_range_prct=reduction_prct)
usable_fit_DMFT = intersect(usable_ch, usable_sp)

usable_naive_ch = LadderDGA.find_usable_interval(real(χchED_ω_naive), reduce_range_prct=reduction_prct)
usable_naive_sp = LadderDGA.find_usable_interval(real(χspED_ω_naive), reduce_range_prct=reduction_prct)


plot(usable_ch .- sP.n_iω .- 1, real(χchED_ω_fit[usable_ch]), label="richardson", c=:green)
plot!(usable_ch .- sP.n_iω .- 1, repeat([0], length(usable_ch)), fillrange=real(χchED_ω_fit[usable_ch]), fillalpha=0.06, c=:green, label=nothing)

plot!(usable_naive_ch .- sP.n_iω .- 1, real(χchED_ω_naive[usable_naive_ch]), label="naive", xlabel="ωₙ", ylabel="χ_ch", c=:orange, title="reduction: $(reduction_prct*100)%")
plot!(usable_naive_ch .- sP.n_iω .- 1, repeat([0], length(usable_naive_ch)), fillrange=real(χchED_ω_naive[usable_naive_ch]), fillalpha=0.3, c=:orange, label=nothing)


LoadError: UndefVarError: impQ_ch not defined

### non-local quantities

In [8]:
bubbleLoc2 = calc_bubble(νGrid2, GImp_fft2, qGridLoc, mP2, sP2)
locQ_sp2 = calc_χ_trilex(impQ_sp2.Γ, bubbleLoc2, qGridLoc, νGrid2, sumHelper_f2, mP2.U, mP2, sP2);
locQ_ch2 = calc_χ_trilex(impQ_ch2.Γ, bubbleLoc2, qGridLoc, νGrid2, sumHelper_f2, -mP2.U, mP2, sP2);
Σ_ladderLoc2 = calc_Σ(locQ_sp2, locQ_ch2, bubbleLoc2, GImp_fft2, FUpDo2, qGridLoc, sumHelper_f2, mP2, sP2)
Σ_ladderLoc2 = Σ_ladderLoc2 .+ mP2.n * mP2.U/2.0;


bubble2 = calc_bubble(νGrid2, GLoc_fft2, qG, mP2, sP2);
nlQ_sp2 = calc_χ_trilex(impQ_sp2.Γ, bubble2, qG, νGrid2, sumHelper_f2, mP2.U, mP2, sP2);
nlQ_ch2 = calc_χ_trilex(impQ_ch2.Γ, bubble2, qG, νGrid2, sumHelper_f2, -mP2.U, mP2, sP2);

nlQ_sp2_nλ = deepcopy(nlQ_sp2)
nlQ_ch2_nλ = deepcopy(nlQ_ch2)
λ_sp2, λ_spch2  = λ_correction!(impQ_sp2, impQ_ch2, FUpDo2, Σ_loc2, Σ_ladderLoc2, nlQ_sp2, nlQ_ch2, bubble2, GLoc_fft2, qG, mP2, sP2)

Σ_ladder2 = calc_Σ(nlQ_sp2, nlQ_ch2, bubble2, GLoc_fft2, FUpDo2, qG, sumHelper_f2, mP2, sP2)
Σ_ladder_corrected2 = Σ_ladder2 .- Σ_ladderLoc2 .+ Σ_loc[1:size(Σ_ladder2,1)];

LoadError: UndefVarError: νGrid2 not defined

In [9]:
# naive:
χch_ω = LadderDGA.kintegrate(qG, nlQ_ch.χ, dim=2)[:,1]
χsp_ω = LadderDGA.kintegrate(qG, nlQ_sp.χ, dim=2)[:,1]
xarr = sort(unique(union(nlQ_ch.usable_ω, nlQ_sp.usable_ω))) 
plot(xarr .- sP.n_iω .- 1, real(χch_ω[xarr]), label="nl χ_ch", legend=:topleft, color=:red)
plot!(twinx(), xarr .- sP.n_iω .- 1, real(χsp_ω[xarr]), label="nl χ_sp")

LoadError: UndefVarError: nlQ_ch not defined

In [10]:
plot(imag.(Σ_ladder_corrected[1:10,1]))

LoadError: UndefVarError: Σ_ladder_corrected not defined

In [11]:
plot(real.(Σ_ladder_corrected[:,1]))

LoadError: UndefVarError: Σ_ladder_corrected not defined